In [1]:
from azureml.core.runconfig import RunConfiguration
from azureml.core import Workspace, Experiment, Run, ScriptRunConfig, Datastore, Dataset
from azureml.core.model import Model
import json
import os
from azureml.core.authentication import AzureCliAuthentication
#from sklearn.externals import joblib
import joblib

In [3]:
#Service Principal Authentication

from azureml.core.authentication import ServicePrincipalAuthentication
import os

base_dir='./../configuration'
config_json = os.path.join(base_dir, "config.json")
with open(config_json, "r") as f:
    config = json.load(f)

workspace_name = config["workspace_name"]
resource_group = config["resource_group"]
subscription_id = config["subscription_id"]
location = config["location"]
    
auth = ServicePrincipalAuthentication(
    tenant_id=config["tenant_id"],
    service_principal_id=config["service_principal_id"],
    service_principal_password=config["service_principal_password"],
)

ws = Workspace.get(
        name=workspace_name,
        subscription_id=subscription_id,
        resource_group=resource_group,        
        auth=auth
    )

print('Workspace name: ' + ws.name, 'Azure region: ' + ws.location, 'Subscription id: ' + ws.subscription_id,'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: ShivaMLservice
Azure region: westus2
Subscription id: 46668180-b0ad-4a49-bed9-88f16f315dce
Resource group: MLGroup


In [ ]:
dataset = Dataset.get_by_name(workspace=ws, name='transaction_ts2013')
df = dataset.to_pandas_dataframe()
df.head()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code B6DQF7WLC to authenticate.


In [ ]:
# Attach Experiment
experiment_name = "arima-localrun"
exp = Experiment(workspace=ws, name=experiment_name)
print(exp.name, exp.workspace.name, sep="\n")

In [ ]:
# Editing a run configuration property on-fly.
run_config_user_managed = RunConfiguration()
run_config_user_managed.environment.python.user_managed_dependencies = True

In [19]:
with open('./scripts/training/transactions_arima.py', 'r') as f:
    print(f.read())



import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sklearn.externals import joblib

from pandas import Grouper
#from pandas.plotting import lag_plot
#from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
#from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
#from sklearn.model_selection import TimeSeriesSplit
#from statsmodels.graphics.gofplots import qqplot
#from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARIMA

from azureml.core import Dataset, Run

run = Run.get_context()
# get input dataset by name
#dataset = run.input_datasets['transaction_ts']

ws = run.experiment.workspace
dataset = Dataset.get_by_name(workspace=ws, name='transaction_ts')

df = datas

In [20]:
print("Submitting an experiment.")

#script_arguments = [dataset.as_named_input('robberies')]

src = ScriptRunConfig(
    source_directory="./scripts",
    script="training/transactions_arima.py",
    run_config=run_config_user_managed,
)
run = exp.submit(src)



Submitting an experiment.


In [21]:
# Shows output of the run on stdout.
run.wait_for_completion(show_output=True, wait_post_processing=True)

RunId: arima-localrun_1593473429_7dbcf5bc
Web View: https://ml.azure.com/experiments/arima-localrun/runs/arima-localrun_1593473429_7dbcf5bc?wsid=/subscriptions/46668180-b0ad-4a49-bed9-88f16f315dce/resourcegroups/MLGroup/workspaces/ShivaMLservice

Streaming azureml-logs/70_driver_log.txt

Entering context manager injector. Current time:2020-06-29T23:30:35.594496
Starting the daemon thread to refresh tokens in background for process with pid = 2677
Entering Run History Context Manager.
Preparing to call script [ training/transactions_arima.py ] with arguments: []
After variable expansion, calling script [ training/transactions_arima.py ] with arguments: []

mean1=631.557791, mean2=729.614038
variance1=231436.602522, variance2=186484.007120
ADF Statistic: -2.910140
p-value: 0.044181
Critical Values:
	1%: -3.453
	5%: -2.872
	10%: -2.572
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1283: RuntimeWarning: invalid value encountered in sqrt
  ret

{'runId': 'arima-localrun_1593473429_7dbcf5bc',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-06-29T23:30:34.06339Z',
 'endTimeUtc': '2020-06-29T23:31:28.553682Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '9830ff7a-9269-44ba-93f1-faf714c926ca',
  'azureml.git.repository_uri': 'git@github.com:us-ocp-ai/TimeSeriesForecastingInPython.git',
  'mlflow.source.git.repoURL': 'git@github.com:us-ocp-ai/TimeSeriesForecastingInPython.git',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': 'ba2e73064099f7b2c08608afc0bf05917b5ab6c8',
  'mlflow.source.git.commit': 'ba2e73064099f7b2c08608afc0bf05917b5ab6c8',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [{'dataset': {'id': '88ea0f06-10e0-4e7f-8679-bb40b535886a'}, 'consumptionDetails': {'type': 'Reference'}}],
 'runDefinition': {'script': 'training/transactions_arima.py',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStor

In [22]:
# Raise exception if run fails
if run.get_status() == "Failed":
    raise Exception(
        "Training on local failed with following run status: {} and logs: \n {}".format(
            run.get_status(), run.get_details_with_logs()
        )
    )

In [23]:
run_id = {}
run_id["run_id"] = run.id
run_id["experiment_name"] = run.experiment.name
with open("./configuration/run_id.json", "w") as outfile:
    json.dump(run_id, outfile)

In [24]:
# Get the latest evaluation result
try:
    with open("./configuration/run_id.json") as f:
        config = json.load(f)
    if not config["run_id"]:
        raise Exception("No new model to register as production model perform better")
except:
    print("No new model to register as production model perform better")
    # raise Exception('No new model to register as production model perform better')
    sys.exit(0)

run_id = config["run_id"]
experiment_name = config["experiment_name"]
exp = Experiment(workspace=ws, name=experiment_name)

run = Run(experiment=exp, run_id=run_id)
names = run.get_file_names
names()
print("Run ID for last run: {}".format(run_id))

Run ID for last run: arima-localrun_1593473429_7dbcf5bc


In [25]:
model_local_dir = "model"
os.makedirs(model_local_dir, exist_ok=True)

# Download Model to Project root directory
model_name = "arima_notebook_model.pkl"
run.download_file(
    name="./outputs/" + model_name, output_file_path="./model/" + model_name
)
print("Downloaded model {} to Project root directory".format(model_name))


Downloaded model arima_notebook_model.pkl to Project root directory


In [26]:
os.chdir("./model")
model = Model.register(
    model_path=model_name,  # this points to a local file
    model_name=model_name,  # this is the name the model is registered as
    tags={"area": "AW Forecasting", "type": "forecasting", "run_id": run_id},
    description="Time series forecasting model for Boston robberies dataset",
    workspace=ws,
)
os.chdir("..")
print(
    "Model registered: {} \nModel Description: {} \nModel Version: {}".format(
        model.name, model.description, model.version
    )
)


Registering model arima_notebook_model.pkl
Model registered: arima_notebook_model.pkl 
Model Description: Time series forecasting model for Boston robberies dataset 
Model Version: 2


In [27]:
# Writing the registered model details to /aml_config/model.json
model_json = {}
model_json["model_name"] = model.name
model_json["model_version"] = model.version
model_json["run_id"] = run_id
with open("./configuration/model.json", "w") as outfile:
    json.dump(model_json, outfile)

In [28]:
model

Model(workspace=Workspace.create(name='ShivaMLservice', subscription_id='46668180-b0ad-4a49-bed9-88f16f315dce', resource_group='MLGroup'), name=arima_notebook_model.pkl, id=arima_notebook_model.pkl:2, version=2, tags={'area': 'AW Forecasting', 'type': 'forecasting', 'run_id': 'arima-localrun_1593473429_7dbcf5bc'}, properties={})

In [29]:
import pickle
import json
import numpy
from azureml.core.model import Model
from statsmodels.tsa.arima_model import ARIMA


def init():
    global model
    from sklearn.externals import joblib

    # load the model from file into a global object
    model_path = Model.get_model_path(model_name="./model/arima_notebook_model.pkl")
    model = joblib.load(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)["data"]
        data = numpy.array(data)
        result=model.forecast(steps=data[0])[0]
        return json.dumps({"result": result.tolist()})
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

In [30]:
init()
step_size = [3]
test_sample = json.dumps({"data": step_size})
test_sample = bytes(test_sample, encoding="utf8")
prediction = run(raw_data=test_sample)
print("Test result: ", prediction)

Test result:  {"result": [796.5203792789346, 916.3628259630898, 848.0511918532666]}
